Перед вами датасет, описывающий поступление детей в колледж.


целевая переменная in_college
Осмотрите данные.
Примените для предобработки пайплайн с прошлого урока.
Решите задачу классификации методами:
- логистической регрессии
- линейной классификации
- деревом решений
- методом опорных векторов
сравните полученные в каждом решении метрики между собой


Попробуйте использовать GridSearch для поиска оптимальных гиперпараметров, перечисленных выше алгоритмов.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

sns.set_style("darkgrid")

In [2]:
df = pd.read_csv("data.csv")
df.head(3)

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True


In [3]:
df.columns

Index(['type_school', 'school_accreditation', 'gender', 'interest',
       'residence', 'parent_age', 'parent_salary', 'house_area',
       'average_grades', 'parent_was_in_college', 'in_college'],
      dtype='object')

In [4]:
df.shape

(1000, 11)

In [5]:
df.dtypes.value_counts()

object     5
int64      2
float64    2
bool       2
dtype: int64

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error

from sklearn.model_selection import GridSearchCV

In [7]:
X = df.drop(["in_college"], axis=1)
Y = df["in_college"]

In [8]:
# Create lists of numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', "bool"]).columns

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [10]:
# Create a numeric and categorical transformer to perform preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [11]:
# Use the ColumnTransformer to apply to the correct features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [42]:
# Append SGDClassifier  to the  preprocessor
SGD_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SGDClassifier ())])

In [43]:
# hyperparameter selection
param_grid_SGD = {'classifier__penalty': ['l1', 'l2']}
SGD_clf_grid = GridSearchCV(SGD_clf, param_grid_SGD, cv = 5, verbose = 0, n_jobs = -1)

In [44]:
# Fit the complete pipeline
SGD_clf_grid.fit(X_train, y_train)
pred_SGD_clf = SGD_clf_grid.predict(X_test)
print(  f1_score(y_test, pred_SGD_clf),
        precision_score(y_test, pred_SGD_clf),
        recall_score(y_test, pred_SGD_clf))

0.8 0.819672131147541 0.78125


In [35]:
# Append logistic regressor to the preprocessor
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

In [36]:
# hyperparameter selection
param_grid_lr = {'classifier__C': np.logspace(-4, 4, 50),'classifier__penalty': ['l1', 'l2']}
lr_grid = GridSearchCV(lr, param_grid_lr, cv = 5, verbose = 0, n_jobs = -1)


In [37]:
# Fit the complete pipeline
lr_grid.fit(X_train, y_train)
pred_lr = lr_grid.predict(X_test)
print(  f1_score(y_test, pred_lr),
        precision_score(y_test, pred_lr),
        recall_score(y_test, pred_lr))

0.8505747126436781 0.8345864661654135 0.8671875


/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
250 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
250 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _c

In [32]:
# Append Tree Classifier to the preprocessor
tree_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

In [33]:
param_grid_tree = {'classifier__max_depth': range(1,11), 'classifier__max_features': range(3,19)}
tree_clf_grid = GridSearchCV(tree_clf, param_grid_tree, cv = 5, verbose = 0, n_jobs = -1)

In [34]:
# Fit the complete pipeline
tree_clf_grid.fit(X_train, y_train)
pred_Tree_clf = tree_clf_grid.predict(X_test)
print(  f1_score(y_test, pred_Tree_clf),
        precision_score(y_test, pred_Tree_clf),
        recall_score(y_test, pred_Tree_clf))

0.8319999999999999 0.8524590163934426 0.8125


In [39]:
# Append Tree Classifier to the preprocessor
SV_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC())])

In [40]:
param_grid_SV_clf= {'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']}
SV_clf_grid = GridSearchCV(SV_clf, param_grid_SV_clf, cv = 5, verbose = 0, n_jobs = -1)

In [41]:
# Fit the complete pipeline
SV_clf_grid.fit(X_train, y_train)
pred_SV_clf = SV_clf_grid.predict(X_test)
print(  f1_score(y_test, pred_SV_clf),
        precision_score(y_test, pred_SV_clf),
        recall_score(y_test, pred_SV_clf))

0.897338403041825 0.8740740740740741 0.921875


/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/sergeys/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py", line 215, in fit
    raise ValueError(
ValueError: 